# Stock Market Analysis
## Marshall Ferguson 9/2021

In [39]:
# Imports

import os
import csv
import numpy as np
import pandas as pd
import yfinance as yf

In [2]:
# Read in data

df = pd.read_csv('data/sp500/AAPL.csv')

df.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close
0,12-12-1980,0.128348,0.128348,469033600,0.128906,0.128348,0.100600
1,15-12-1980,0.121652,0.122210,175884800,0.122210,0.121652,0.095352
2,16-12-1980,0.112723,0.113281,105728000,0.113281,0.112723,0.088353
3,17-12-1980,0.115513,0.115513,86441600,0.116071,0.115513,0.090540
4,18-12-1980,0.118862,0.118862,73449600,0.119420,0.118862,0.093165


As an aside, the dates in the Date Series are in DD-MM-YYY format

This data is set up differently than I would like. It is organized into folders based on whether the company is part of NASDAQ, NYSE, or S&P500. Within those folders are seperate CSV files for each company. I would like to combine these seperate files into one master dataset. My thought process is that I can turn each CSV into a dict, and then turn it into one giant df from combining each dict.

In [3]:
# Trial run converting AAPL csv to a list of dicts

AAPL = []

with open('data/sp500/AAPL.csv', newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        row['Ticker'] = 'AAPL'
        AAPL.append(row)

AAPL[0]

{'Date': '12-12-1980',
 'Low': '0.1283479928970337',
 'Open': '0.1283479928970337',
 'Volume': '469033600',
 'High': '0.1289059966802597',
 'Close': '0.1283479928970337',
 'Adjusted Close': '0.10060006380081177',
 'Ticker': 'AAPL'}

In [4]:
# Convert all rows in all files in sp500 directory to dicts
# and append said dicts to one list, after adding Ticker to dict

sp500 = []

for file in os.listdir('data/sp500'):
    with open(f'data/sp500/{file}', newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            file_name = os.path.splitext(file)
            row['Ticker'] = file_name[0]
            sp500.append(row)

In [5]:
sp500[0]

{'Date': '18-11-1999',
 'Low': '28.612302780151367',
 'Open': '32.54649353027344',
 'Volume': '62546380',
 'High': '35.765380859375',
 'Close': '31.473533630371094',
 'Adjusted Close': '27.135984420776367',
 'Ticker': 'A'}

In [6]:
sp500[-1]

{'Date': '30-08-2021',
 'Low': '204.0050048828125',
 'Open': '203.82000732421875',
 'Volume': '288506.0',
 'High': '206.97999572753906',
 'Close': '205.75',
 'Adjusted Close': '205.75',
 'Ticker': 'ZTS'}

In [7]:
# Convert list of sp500 dicts to df

sp500_df = pd.DataFrame(sp500)

In [8]:
sp500_df.head()

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
0,18-11-1999,28.612302780151367,32.54649353027344,62546380,35.765380859375,31.473533630371094,27.135984420776367,A
1,19-11-1999,28.47818374633789,30.713518142700195,15234146,30.75822639465332,28.880544662475586,24.900341033935547,A
2,22-11-1999,28.65700912475586,29.551143646240234,6577870,31.473533630371094,31.473533630371094,27.135984420776367,A
3,23-11-1999,28.612302780151367,30.400571823120117,5975611,31.205293655395508,28.612302780151367,24.669065475463867,A
4,24-11-1999,28.612302780151367,28.701717376708984,4843231,29.998212814331055,29.372318267822266,25.324344635009766,A


In [9]:
sp500_df.tail()

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
3221722,24-08-2021,204.49000549316406,206.08999633789062,97870000.0,206.9600067138672,204.99000549316406,204.99000549316406,ZTS
3221723,25-08-2021,202.97999572753906,204.38999938964844,78070000.0,205.0800018310547,204.42999267578125,204.42999267578125,ZTS
3221724,26-08-2021,203.36000061035156,204.8000030517578,95760000.0,205.49000549316406,204.02000427246094,204.02000427246094,ZTS
3221725,27-08-2021,203.22999572753906,204.67999267578125,85530000.0,206.19000244140625,203.82000732421875,203.82000732421875,ZTS
3221726,30-08-2021,204.0050048828125,203.82000732421875,288506.0,206.97999572753906,205.75,205.75,ZTS


Now that we have one dataset for all the S&P500 data, let's do some data cleaning...

In [10]:
# What types of data are in the dataframe?

sp500_df.dtypes

Date              object
Low               object
Open              object
Volume            object
High              object
Close             object
Adjusted Close    object
Ticker            object
dtype: object

We need to convert the Series' that represent stock prices into numeric values

In [11]:
numb_convert = ['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close']

for series in numb_convert:
    sp500_df[series] = pd.to_numeric(sp500_df[series])

In [12]:
sp500_df.dtypes

Date               object
Low               float64
Open              float64
Volume            float64
High              float64
Close             float64
Adjusted Close    float64
Ticker             object
dtype: object

Now that the numeric data is treated as numerics by Python, let's do some analysis...

### Which company has the highest ATH(All Time High)?

In [13]:
# Find highest value in High Series
higest_ath = sp500_df['High'].max()
# Find index of highest value in High Series
highest_ath_index = sp500_df['High'].idxmax()
# Find stock ticker for highest value
ticker_highest_ath = sp500_df.iloc[highest_ath_index]['Ticker']
# Find date at which the stock was at ATH
date_highest_ath = sp500_df.iloc[highest_ath_index]['Date']

print(f'The company with the highest ATH in the dataset is {ticker_highest_ath} with a price of {higest_ath} on {date_highest_ath}.')

The company with the highest ATH in the dataset is BRK-A with a price of 445000.0 on 07-05-2021.


### Which company has the lowest ATL(All Time Low)?

In [14]:
# Find lowest value in Low Series
lowest_atl = sp500_df['Low'].min()
# Find index of lowest value in Low Series
lowest_atl_index = sp500_df['Low'].idxmin()
# Find stock ticker for lowest value
ticker_lowest_atl = sp500_df.iloc[lowest_atl_index]['Ticker']
# Find date at which the stock was at ATL
date_lowest_atl = sp500_df.iloc[lowest_atl_index]['Date']

print(f'The company with the lowest ATL in the dataset is {ticker_lowest_atl} with a price of {lowest_atl} on {date_lowest_atl}.')

The company with the lowest ATL in the dataset is NEOG with a price of 0.0 on 05-08-1993.


It seems a little fishy that any stock could have a price of $0.00. I am wondering if some of the trailing decimal places got truncated. Let's look deeper into the NEOG ticker to find out more. 

In [15]:
sp500_df.iloc[lowest_atl_index]

Date              05-08-1993
Low                      0.0
Open                     0.0
Volume               14200.0
High                     0.0
Close                    0.0
Adjusted Close           0.0
Ticker                  NEOG
Name: 2130882, dtype: object

In [16]:
neog_df = sp500_df[sp500_df['Ticker'] == 'NEOG']

neog_df

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
2129883,23-08-1989,0.444556,0.444556,966800.0,0.455669,0.444556,0.444556,NEOG
2129884,24-08-1989,0.500125,0.500125,707000.0,0.511239,0.500125,0.500125,NEOG
2129885,25-08-1989,0.466783,0.489011,768800.0,0.489011,0.466783,0.466783,NEOG
2129886,28-08-1989,0.466783,0.466783,359800.0,0.489011,0.466783,0.466783,NEOG
2129887,29-08-1989,0.455669,0.466783,230200.0,0.477897,0.455669,0.455669,NEOG
...,...,...,...,...,...,...,...,...
2137946,24-08-2021,43.509998,43.790001,22900000.0,43.959999,43.590000,43.590000,NEOG
2137947,25-08-2021,43.009998,43.490002,16610000.0,43.770000,43.119999,43.119999,NEOG
2137948,26-08-2021,42.590000,43.150002,16570000.0,43.150002,42.610001,42.610001,NEOG
2137949,27-08-2021,42.740002,42.740002,25900000.0,43.900002,43.750000,43.750000,NEOG


In [17]:
neog_df[neog_df['Date'] == '05-08-1993']

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
2130882,05-08-1993,0.0,0.0,14200.0,0.0,0.0,0.0,NEOG


What about the day before or the day after?

In [18]:
neog_df[neog_df['Date'] == '04-08-1993']

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
2130881,04-08-1993,0.216721,0.222278,241000.0,0.266733,0.266733,0.266733,NEOG


In [19]:
neog_df[neog_df['Date'] == '06-08-1993']

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
2130883,06-08-1993,0.216721,0.244506,218200.0,0.244506,0.222278,0.222278,NEOG


The days before and after the day of the ATL both have values for the Low and High Series', and both days have extended decimal values where there does not seem to be any values at all for 05-08-1993 for the prices. I am unsure as to what is going on here, but do not want to spend too much time on this specific case. Having said that, I do want to know if there are other 0 values in the dataset so I can look into why.

In [20]:
sp500_df[sp500_df['High'] == 0]

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
2130882,05-08-1993,0.0,0.0,14200.0,0.0,0.0,0.0,NEOG


In [21]:
sp500_df[sp500_df['Low'] == 0]

,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker
2130882,05-08-1993,0.0,0.0,14200.0,0.0,0.0,0.0,NEOG


It looks like this row is the only one with 0 values for the prices. Strange, but relatively not important. Let's move on. 

Let's look at which company had the best percentage gain over the course of the company's biggest peak and lowest valley. To do this, we need to go through each company and find their lowest price and their highest price, subtract the low price from the high price, and divide that difference by the the low price, and then multiply by 100 to get the percentage.

In [22]:
# For each ticker, find lowest price

lowest = sp500_df.groupby(['Ticker'])['Low'].min()

lowest

Ticker
A        7.510730
AAL      1.450000
AAP     12.330000
AAPL     0.049107
ABBV    33.330002
          ...    
XYL     22.430000
YUM      4.234813
ZBH     24.700001
ZION     1.604167
ZTS     28.139999
Name: Low, Length: 425, dtype: float64

In [23]:
# For each ticker, find highest price

highest = sp500_df.groupby(['Ticker'])['High'].max()

highest

Ticker
A       177.050003
AAL      63.270000
AAP     217.690002
AAPL    153.250000
ABBV    125.860001
           ...    
XYL     137.184998
YUM     135.770004
ZBH     180.360001
ZION     88.559998
ZTS     208.559998
Name: High, Length: 425, dtype: float64

In [24]:
peak_valley_df = pd.DataFrame([lowest, highest])

peak_valley_df

Ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADI,...,WYNN,XEL,XLEFF,XLNX,XOM,XYL,YUM,ZBH,ZION,ZTS
Low,7.510730,1.45,12.330000,0.049107,33.330002,2.468750,2.00,0.478711,11.300000,0.592,...,10.760000,3.812500,0.046,0.791667,1.558594,22.430000,4.234813,24.700001,1.604167,28.139999
High,177.050003,63.27,217.690002,153.250000,125.860001,128.869995,459.75,128.539993,338.648895,175.250,...,249.309998,76.440002,0.372,159.300003,104.760002,137.184998,135.770004,180.360001,88.559998,208.559998


In [25]:
peak_valley_df = peak_valley_df.transpose()

peak_valley_df

,Low,High
Ticker,,
A,7.510730,177.050003
AAL,1.450000,63.270000
AAP,12.330000,217.690002
AAPL,0.049107,153.250000
ABBV,33.330002,125.860001
...,...,...
XYL,22.430000,137.184998
YUM,4.234813,135.770004
ZBH,24.700001,180.360001


In [26]:
peak_valley_df.index

Index(['A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADI',
       ...
       'WYNN', 'XEL', 'XLEFF', 'XLNX', 'XOM', 'XYL', 'YUM', 'ZBH', 'ZION',
       'ZTS'],
      dtype='object', name='Ticker', length=425)

I may have created to DataFrame incorrectly, or it may be how pandas works, but I want tickers to be a Series in the DataFrame rather than the index.

In [27]:
peak_valley_df['Ticker'] = peak_valley_df.index

In [28]:
peak_valley_df

,Low,High,Ticker
Ticker,,,
A,7.510730,177.050003,A
AAL,1.450000,63.270000,AAL
AAP,12.330000,217.690002,AAP
AAPL,0.049107,153.250000,AAPL
ABBV,33.330002,125.860001,ABBV
...,...,...,...
XYL,22.430000,137.184998,XYL
YUM,4.234813,135.770004,YUM
ZBH,24.700001,180.360001,ZBH


In [29]:
peak_valley_df = peak_valley_df.reset_index(drop=True)

That is much better. Now let's go through each row, caluclate the percentage increase between low and high, and find the highest percentage from that.

In [30]:
peak_valley_df['Percentage Increase'] = (((peak_valley_df['High'] - peak_valley_df["Low"]) / peak_valley_df['Low']) * 100)

In [31]:
peak_valley_df

,Low,High,Ticker,Percentage Increase
0,7.510730,177.050003,A,2257.294271
1,1.450000,63.270000,AAL,4263.448164
2,12.330000,217.690002,AAP,1665.531255
3,0.049107,153.250000,AAPL,311973.633651
4,33.330002,125.860001,ABBV,277.617743
...,...,...,...,...
420,22.430000,137.184998,XYL,511.613891
421,4.234813,135.770004,YUM,3106.044693
422,24.700001,180.360001,ZBH,630.202409
423,1.604167,88.559998,ZION,5420.623361


In [32]:
# Find Ticker with highest percentage increase

peak_valley_df['Percentage Increase'].max()

inf

It seems as though we have an infinite value in the percentage increase column. This is likly due to the GEOM ticker's low being 0.00 earlier. Looks like the issue has more importance than I gave it credit for. To remedy this, we will replace all infinite values with NaN's.

In [33]:
peak_valley_df = peak_valley_df.replace([np.inf, -np.inf], np.nan)

Let's see if that worked

In [34]:
# Find Ticker with highest percentage increase

peak_valley_df['Percentage Increase'].max()

112500002741.9891

Looks like that did it. Let's continue the analysis.

In [35]:
# Find Ticker with highest percentage increase

highest_pct = peak_valley_df['Percentage Increase'].max()
highest_pct_index = peak_valley_df['Percentage Increase'].idxmax()
highest_pct_ticker = peak_valley_df.iloc[highest_pct_index]['Ticker']

In [36]:
print(f'The company with the largest percent increase from lowest to highest price is {highest_pct_ticker} with an increase of {highest_pct}%.')

The company with the largest percent increase from lowest to highest price is INTH with an increase of 112500002741.9891%.


For someone unfamiliar with the market (and even those familiar with it) it can be difficult to know what ticker represents what company. I have just installed and imported the yfinance library to pull some info about the companies into which we are looking. 

Since we'll be doing this often, let's create a function to handle the brunt of the labor. 

In [45]:
def find_company_name(ticker_symbol):
    company = yf.Ticker(ticker_symbol)
    company_info = company.info
    company_name = company_info['shortName']
    return company_name

In [57]:
find_company_name('AAPL')

'Apple Inc.'

In [47]:
find_company_name('MSFT')

'Microsoft Corporation'

We have created the function and tested it. It works, so now let's put it into practice. 

In [58]:
# Rerun print statement with company name instead of stock ticker

print(f'The company with the largest percent increase from lowest to highest price is {find_company_name(highest_pct_ticker)} with an increase of {highest_pct}%.')

The company with the largest percent increase from lowest to highest price is INNOTECH CORP. with an increase of 112500002741.9891%.


Much better. To be honest, I thought the INTH ticker represented Intel, but apparantly not.